In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(r"C:\Users\sivak\Excel_files\Assessment_Data.xlsx")
df

,firstName,lastName,updatedOn,assessmentTitle,assessmentID,totalNumberQuestions,questionsDone,overallScore,GRADE
0,Little Jimmy,Jr.,2023-11-27 16:40:13.329,Interpreting the weather,NaN,0,0,0.000000,1
1,Learnt,Africa,2023-08-07 08:46:48.538,Basic Addition Part One,NaN,0,0,0.000000,1
2,Collins,Kesuibai,2023-07-06 16:51:03.625,Morning Greetings,NaN,0,0,0.000000,1
3,Learnt,Africa,2022-04-22 20:19:30.877,NaN,e2qxi19gMDjRHzmJ0EuJ,2,2,1.000000,1
4,Mark,Moriama,2022-04-22 21:48:30.971,NaN,WQIiM5iXz8tqUwyi6n1K,2,2,0.500000,1
...,...,...,...,...,...,...,...,...,...
90,Collins,Kesuibai,2022-08-24 18:35:11.584,Counting Numbers 1-10,N6EFBOe2aCeglrUR2qUy,7,11,1.800000,1
91,Learnt,Africa,2023-12-27 10:57:42.419,Counting Numbers 20 - 30,N6EFBOe2aCeglrUR2qUy,7,7,1.000000,1
92,Learnt,Africa,2024-05-05 14:44:12.761,Counting Numbers 1-10,N6EFBOe2aCeglrUR2qUy,7,7,1.000000,1
93,Mark,Moriama,2022-06-20 10:07:53.693,Counting Numbers 1-10,N6EFBOe2aCeglrUR2qUy,7,7,0.571429,1


In [79]:
df = df.dropna(subset=['assessmentID', 'assessmentTitle']).copy()

# Create unique identifier for students and rename columns as needed
df['student_name'] = df['firstName'].fillna('') + ' ' + df['lastName'].fillna('')
df_for_model = df[['student_name', 'assessmentID', 'overallScore', 'assessmentTitle']]

In [96]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras import layers

In [98]:
df_for_model['student_name'] = df_for_model['student_name'].astype(str)
df_for_model['assessmentID'] = df_for_model['assessmentID'].astype(str)
df_for_model['overallScore'] = df_for_model['overallScore'].astype(float)

C:\Users\sivak\AppData\Local\Temp\ipykernel_8720\2481267760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_model['student_name'] = df_for_model['student_name'].astype(str)
C:\Users\sivak\AppData\Local\Temp\ipykernel_8720\2481267760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_model['assessmentID'] = df_for_model['assessmentID'].astype(str)
C:\Users\sivak\AppData\Local\Temp\ipykernel_8720\2481267760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [100]:
# Define unique mappings for student_name and assessmentID
student_name_lookup = layers.StringLookup(vocabulary=df_for_model['student_name'].unique(), mask_token=None)
assessment_id_lookup = layers.StringLookup(vocabulary=df_for_model['assessmentID'].unique(), mask_token=None)
assessment_id_lookup_inv = layers.StringLookup(vocabulary=df_for_model['assessmentID'].unique(), mask_token=None, invert=True)

In [102]:
# Convert data to a TensorFlow dataset and map names to integer IDs
tf_dataset = tf.data.Dataset.from_tensor_slices({
    'student_name': student_name_lookup(df_for_model['student_name'].values),
    'assessmentID': assessment_id_lookup(df_for_model['assessmentID'].values),
    'score': df_for_model['overallScore'].values
}).batch(128)

In [104]:
class CFModel(tfrs.Model):
    def __init__(self, unique_students, unique_assessments):
        super().__init__()
        embedding_dim = 32
        # Embedding layers
        self.user_embedding = layers.Embedding(input_dim=unique_students, output_dim=embedding_dim)
        self.item_embedding = layers.Embedding(input_dim=unique_assessments, output_dim=embedding_dim)
        
        # Precompute candidate embeddings
        candidate_embeddings = self.item_embedding(tf.range(unique_assessments))
        
        # Retrieval task with precomputed embeddings
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=candidate_embeddings)
        )

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_embedding(features["student_name"])
        item_embeddings = self.item_embedding(features["assessmentID"])
        return self.task(user_embeddings, item_embeddings, compute_metrics=not training)

In [106]:
# Get unique counts for embedding layers
unique_students = len(student_name_lookup.get_vocabulary())
unique_assessments = len(assessment_id_lookup.get_vocabulary())

In [108]:
model = CFModel(unique_students, unique_assessments)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(tf_dataset, epochs=5)

Epoch 1/5


C:\Users\sivak\anaconda3\Lib\site-packages\keras\src\layers\layer.py:372: UserWarning: `build()` was called on layer 'retrieval_17', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - loss: 404.9844 - regularization_loss: 0.0000e+00 - total_loss: 404.9844
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - loss: 402.3571 - regularization_loss: 0.0000e+00 - total_loss: 402.3571
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_

In [110]:
assessment_titles = df_for_model[['assessmentID', 'assessmentTitle']].drop_duplicates().set_index('assessmentID')['assessmentTitle'].to_dict()

def get_recommendations(model, student_name, top_k=10):
    student_id = student_name_lookup(tf.constant([student_name]))
    user_embedding = model.user_embedding(student_id)
    
    scores = tf.matmul(user_embedding, model.item_embedding.embeddings, transpose_b=True)
    top_items = tf.argsort(scores, axis=1, direction="DESCENDING")[:, :top_k]
    
    recommended_assessment_ids = assessment_id_lookup_inv(top_items.numpy().flatten())

    
    recommended_assessments = []
    for assessment_id in recommended_assessment_ids:
        title = assessment_titles.get(str(assessment_id.numpy().decode('utf-8')), "Unknown Title")
        recommended_assessments.append(title)
    
    return recommended_assessments

In [114]:
student_name = 'John Doe'
recommended_assessments = get_recommendations(model, student_name)
print("Recommended Assessment Titles:", recommended_assessments)


Recommended Assessment Titles: ['Alphabets Lesson 1 (A-M)', 'Courtesy Words', 'Alphabets Lesson 2 (Upper and Lower Case)', 'Our solar system', 'Counting Numbers 10 - 20', 'Evening Greetings', 'Counting Numbers 20 - 30', 'Afternoon Greetings', 'Morning Greetings', 'Unknown Title']
